In [1]:
# import libraries and modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import gmaps
import time
import scipy.stats as scistats
import random
import requests
import json
from datetime import datetime
from config import api_key
from config import gkey

In [32]:
# Kristina: Module to generate list of movie IDs
def highest_revenue(x=1960,y=2010):
    year_df = pd.DataFrame()
    decade_df = pd.DataFrame()
    for year in range(x,y): 
        time.sleep(0.25)
        response = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' 
                            +  api_key + '&primary_release_year=' + str(year) + '&sort_by=revenue.desc&language=en-US')

        #store parsed json response
        highest_revenue = response.json() 
        # highest_revenue
        highest_revenue_films = highest_revenue['results']
        data_df = pd.DataFrame(highest_revenue_films)

        year_df[str(year)] = data_df['id']

    for z in range(0,int((y-x)/10)):    
        decade_list = []
        for year in range(x+(z*10),(x+10)+(z*10)):
            decade_list.append(year_df[str(year)])
        flat_decade_list = [item for x in decade_list for item in x]
        decade_df[str(year-9)] = flat_decade_list
    return decade_df

In [40]:
# Evan: Module to get list of people from cast
def top2people(movies):
    
    peopleDict = {}

    # loop through each movie in each column
    for column in movies.iteritems():
        peopleDict[column[0]] = []
        for movie in column[1]:
#             print(movie)
            # request credits list for movie
            cast_url = f'https://api.themoviedb.org/3/movie/{movie}/credits?api_key={api_key}'
            data = requests.get(cast_url).json()
            
            time.sleep(0.1)

            if data['cast']:
                # get list of cast IDs
                idList = {}
                for n,x in enumerate(data['cast']):
                    idList[n] = x['cast_id']

                # determine top 2 IDs
                idDF = pd.DataFrame.from_dict(idList, orient='index', columns=['cast_id'])
                idDF.sort_values('cast_id', inplace=True)
                idDF.reset_index(drop=False, inplace=True)

                # get person ID for top 2
                person1 = data['cast'][idDF.iloc[0]['index']]['id']
#                 print(f"    {person1}")
                if len(data['cast']) > 1:
                    person2 = data['cast'][idDF.iloc[1]['index']]['id']
#                     print(f"    {person2}")

                # check for uniqueness 
                if person1 not in peopleDict[column[0]]:
                    peopleDict[column[0]].append(person1)
                if len(data['cast']) > 1:
                    if person2 not in peopleDict[column[0]]:
                        peopleDict[column[0]].append(person2)
                
    # convert dictionary of people IDs to DataFrame
    peopleDF = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in peopleDict.items()]))
    return peopleDF

In [51]:
movies = highest_revenue(1960,2010)

In [52]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
1960    200 non-null int64
1970    200 non-null int64
1980    200 non-null int64
1990    200 non-null int64
2000    200 non-null int64
dtypes: int64(5)
memory usage: 7.9 KB


In [53]:
people = top2people(movies)


967
    2090
    3359
539
    7301
    7302
284
    3151
    4090
1941
    3636
    2639
439
    5676
    5961
22013
    13784
    14500
15788
    3663
    78778
43039
    29902
    14227
11209
    4165
    12149
18973
    3663
    5833
25767
    20141
    10158
966
    14528
    3265
23439
    1905
    100989
11656
    2201
    70359
11167
    6251
    68422
29259
    24495
    103397
2216
    12647
    22748
59020
    32059
    18966
86297
    6352
    7301
116385
    2753
    7632
12230
    8229
    22602
1725
    2769
    6679
10911
    8487
    14261
36362
    30551
    14012
821
    12147
    13784
164
    1932
    1933
11536
    11492
    3149
990
    3636
    14882
2160
    12308
    3360
248
    3380
    3381
18647
    3084
    9857
28284
    45363
    30155
430
    5788
    5789
28501
    1905
    62881
31522
    24299
    18197
84249
    1534583
    1534584
40894
    18735
    8237
101778
    235260
    511505
63680
    39644
    18809
195576
    230266
    90212
982
    434

    4483
    3359
21955
    62036
    88097
53566
    54384
    587152
11
    2
    3
840
    3037
    1650
11009
    8891
    67787
691
    10222
    10458
11006
    16475
    67764
14741
    3037
    19131
11319
    64930
    44714
5902
    21605
    3085
24032
    90802
    80625
703
    1243
    3092
12535
    14639
    29803
11586
    37366
    5341
7227
    3151
    30123
11590
    3636
    8260
17689
    190
    66223
12262
    73781
    73782
37749
    3092
    4514
19050
    17183
    21416
30315
    1748
    99837
12637
    66776
    380
1924
    20006
    3084
579
    6355
    8607
621
    8891
    8892
8469
    7171
    21561
16214
    16475
    26715
923
    15069
    15070
11778
    380
    4690
6081
    12446
    14503
35976
    16475
    6844
11455
    11159
    63208
11327
    69010
    69011
123
    16270
    16271
36361
    6837
    40176
10766
    8252
    30123
11850
    55636
    69054
38731
    20362
    27554
12611
    2090
    11147
9584
    10823
    46597
249

    206
    8447
628
    287
    500
9331
    5293
    10427
8984
    3392
    3416
680
    62
    8891
2164
    6856
    22138
9359
    2461
    1038
2636
    16483
    4430
4476
    287
    4173
10395
    514
    290
193
    2387
    2388
306
    776
    1210
10731
    4038
    2176
862
    31
    12898
1572
    16940
    62
568
    31
    2053
710
    517
    48
10530
    65529
    2461
414
    5576
    2176
807
    287
    192
8839
    56057
    10713
9804
    1269
    59350
8844
    2157
    205
9598
    58136
    6199
9273
    206
    3547
197
    2461
    2462
6950
    4483
    14343
949
    1158
    380
2064
    18277
    8984
688
    5064
    190
9909
    1160
    10477
63
    62
    289
8963
    193
    5292
664
    9994
    2053
954
    500
    10127
9802
    2963
    228
11674
    515
    8447
3595
    2461
    33
9390
    500
    9777
2300
    23678
    4201
9268
    1100
    3085
409
    5469
    1137
11000
    2157
    193
4232
    15234
    9206
819
    4724
    8289
11

In [50]:
people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259 entries, 0 to 258
Data columns (total 2 columns):
1920    246 non-null float64
1930    259 non-null int64
dtypes: float64(1), int64(1)
memory usage: 4.1 KB


In [55]:
# Aruna: Module to generate DataFrame of people with birth state and places
def birthplaces(people)
    birthplace = {}
    for column in people.iteritems():
        birthplace[column[0]] = []
        for x in range(column[1].count()):
            time.sleep(0.25)
            ids = column[1][x]
    #         print (ids)
    #         print(type(ids))
            if ids != 'nan':
                url = f'https://api.themoviedb.org/3/person/{ids}?api_key={api_key}'
                response = requests.get(url).json()
                #names.append(response['name'])

            #convert dictionary of people IDs to DataFrame
                if response['place_of_birth'] != 'null':
                     birthplace[column[0]].append(response['place_of_birth']) 
    birthDF = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in birthplace.items()]))
    return birthDF

,1960,1970,1980,1990,2000
0,"Amsterdam, New York, USA","Pound Ridge, New York","Concord, California, USA","Houston, Texas, USA","Syracuse, New York, USA"
1,"Dorking, Surrey, England, UK",Los Angeles - California - USA,"Chicago, Illinois, USA","Roswell, New Mexico, US","Glenrothes, Fife, Scotland, UK"
2,"New York City, New York, USA","New York City, New York, USA",New York City - New York - USA,"New York City, New York, USA","Wellington, New Zealand"
3,"Merced, California, USA","Steubenville, Ohio, USA","Yellowknife, Northwest Territories, Canada","Newark, New Jersey, USA","San Juan, Puerto Rico"
4,"Newton, Massachusetts, USA","Wise, Virginia, USA","Ottawa, Ontario, Canada","Smyrna, Georgia, USA","Concord, California, USA"


In [68]:
# mock data
mockPlaces = pd.DataFrame({'place':['Seattle, WA, USA',
                                   'Portland, OR, USA',
                                   'Los Angeles, CA, USA',
                                   'Kansas City, MO, USA',
                                   'Toronto, CAN']
                       })
def placesMap(places):
    
    for column in places.iteritems():
        # location dictionary
        placesDict = {'lat':[], 'lng':[]}
        for x in range(column[1].count()):
            
            place = column[1][x]
            
            if place != None:
                # format location data
                place.replace(" ", "%20")

                # maps url
                mapUrl = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?inputtype=textquery'



                # iterate through places dataframe

                # pull city/state variables from the row

                # select fields with lat/long data
                fields = 'id,geometry'
                # build query url
                url = f'{mapUrl}&input={place}&fields={fields}&key={gkey}'
                # generate json data
                data = requests.get(url).json()
                # grab lat and long data from json
                lat = data['candidates'][0]['geometry']['location']['lat']
                lng = data['candidates'][0]['geometry']['location']['lng']
                # add data to dictionary
                placesDict['lat'].append(lat)
                placesDict['lng'].append(lng)

        # location dataframe
        locationDF = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in placesDict.items()]))

        # Configure gmaps with API key
        gmaps.configure(api_key=gkey)

        # Store 'Lat' and 'Lng' into  locations 
        locations = locationDF[['lat','lng']]
        
        # create list of equal weights
        weights = [1] * len(locations)

        # Create a pins layer
#             fig = gmaps.figure()

#             artist_layer = gmaps.symbol_layer(
#                 locations, fill_color='rgba(0, 255, 255, 0.8)',
#                 stroke_color='rgba(0, 255, 255, 0.2)', scale=2
#             )


#             fig.add_layer(artist_layer)

        # create a heatmap layer
        fig = gmaps.figure()
        heat_layer = gmaps.heatmap_layer(locations, weights=weights,
                                         dissipating=False, max_intensity=100,
                                         point_radius = 1)

        # Adjust heat_layer setting to help with heatmap dissipating on zoom
        heat_layer.dissipating = False
        heat_layer.max_intensity = 100
        heat_layer.point_radius = 1

        fig.add_layer(heat_layer)

    return fig

In [69]:
placesMap(birthDF.head(50))

AttributeError: 'Figure' object has no attribute 'savefig'

In [66]:
birthDF.sample(20)

,1960,1970,1980,1990,2000
8,"Fontana Liri, Latium, Italy","Linton, Indiana, USA","New York City, New York, USA","Thal, Styria, Austria","Tulsa, Oklahoma, USA"
18,"Philadelphia, Pennsylvania, USA","New York City, New York, USA","East Chicago, Indiana, USA","Johnstown, Pennsylvania, USA","Santa Ana, California, USA"
189,"Bramhall, Cheshire, England, UK","Quitman, Texas, USA","Lightning Ridge, New South Wales, Australia","Alexandria, Virginia, USA","Brooklyn, New York, USA"
90,"Bronx, New York, USA","Towson, Maryland, USA","Brooklyn, New York, USA",Philadelphia - Pennsylvania - USA,"Houston, Texas, United States"
280,None,"Pueblo, Colorado, USA",NaN,NaN,NaN
58,"New York City, New York, U.S.","Studio City, California, USA","Dorking, Surrey, England, UK","Montreal, Quebec, Canada","Pompton Plains, New Jersey, USA"
242,"New York City, New York USA","Los Angeles, California, USA","Istanbul, Turkey","Pompton Plains, New Jersey, USA","Rockwood, Tennessee, USA"
53,"Tokyo, Japan",None,"Istanbul, Turkey","Long Beach, Long Island, New York, USA","Margam, Port Talbot, Glamorgan, Wales, UK"
266,None,"Yellowknife, Northwest Territories, Canada","Huntington, New York, USA",NaN,NaN
158,"Lyon, France","San Mateo, California, USA","Whitewater, Wisconsin, USA","Yonkers, New York, USA","Owensboro, Kentucky, USA"
